In [1]:
import numpy as np

In [2]:
x0=np.array([[0,0],[0,1],[1,0],[1,1]])#xor input
y0=np.array([[0],[1],[1],[0]])#xor output


LEARNING_RATE=0.001

global pm_output, pm_hidden

In [3]:
def init_model_hidden1():
    global pm_output, pm_hidden
    pm_hidden = alloc_param_pair([2, 10])#input,hidden  weight shape:2x10, bias:1x10
    pm_output = alloc_param_pair([10, 1])#hidden,output  weight shape:10x1, bias:1x1
    
def alloc_param_pair(shape):
    weight = np.random.rand(shape[0],shape[1])# shape[0] x shape[1] size의 0~1사이 난수 생성
    bias = np.zeros(shape[-1])
    return {'w':weight, 'b':bias} #딕셔너리 형태로 반환



In [4]:
def forward_neuralnet_hidden1(x): #순전파
    global pm_output, pm_hidden
    x=x.reshape(1,2)
    hidden = relu(np.matmul(x, pm_hidden['w']) + pm_hidden['b']) # 1x2 * 2x10 = 1 x 10 
    output = np.matmul(hidden, pm_output['w']) + pm_output['b'] #1x10 * 10x1 = 1x1
    
    return output, [x, hidden]

def relu(x):
    return np.maximum(x, 0) #배열 요소의 요소별 최대값 0 if x<0 else x




In [5]:
def backprop_neuralnet_hidden1(G_output, aux):
    global pm_output, pm_hidden
    
    x, hidden = aux #출력층,은닉측 가중치 편미분 정보
    #ㅡㅡㅡㅡㅡ출력층 역전파 // 손실기울기 계산ㅡㅡㅡㅡㅡ
    g_output_w_out = hidden.transpose()#출력부의 입력(히든층의 출력).transpose()
    G_w_out = np.matmul(g_output_w_out, G_output)#(입력)T x (Loss에서 들어온 미분값) ->> 가중치 손실 미분값            
    G_b_out = np.sum(G_output, axis=0)# 편향벡터 손실 미분값 ==> 열방향 합                   
    #ㅡㅡㅡㅡㅡ히든층 역전파ㅡㅡㅡㅡㅡ
    g_output_hidden = pm_output['w'].transpose()#출력층의 가중치.transpose // 가중치 업데이트 하기전에 가져오기             
    G_hidden = np.matmul(G_output, g_output_hidden)#입력(히든층의 출력)에 대한 손실 미분값     
    #ㅡㅡㅡㅡㅡ출력층 업데이트ㅡㅡㅡㅡㅡ
    pm_output['w'] -= LEARNING_RATE * G_w_out  # 가중치 업데이트           
    pm_output['b'] -= LEARNING_RATE * G_b_out  # 편향 업데이트
    #ㅡㅡㅡㅡㅡ히든층 역전파ㅡㅡㅡㅡㅡ
    G_hidden = G_hidden * relu_derv(hidden) #ReLU 도함수 1or0 반환 -->g_hidden은 자기자신을 반환하던가 0이 반환됨
    g_hidden_w_hid = x.transpose()# 최초 입력 .transpose()                        
    G_w_hid = np.matmul(g_hidden_w_hid, G_hidden) #(입력)T x (출력층에서 들어온 미분값) ->> 가중치 손실 미분값           
    G_b_hid = np.sum(G_hidden, axis=0) #편향벡터 손실 미분값 ==> 열방향 합                       
    #ㅡㅡㅡㅡㅡ히든층 업데이트ㅡㅡㅡㅡㅡ
    pm_hidden['w'] -= LEARNING_RATE * G_w_hid   # 가중치 업데이트              
    pm_hidden['b'] -= LEARNING_RATE * G_b_hid   # 편향 업데이트             
    
def relu_derv(y): #ReLU의 도함수는 1 if y>0 else 0 이어야 하지만
    return np.sign(y) #y의 자리에는 활성화 함수 ReLU를 지난 hidden이 들어감으로 np.sign이 ReLU의 도함수 역할을 한다 

In [6]:
def forward_postproc(output, y): #MSE 
    diff = output - y #출력과 정답과의 편차
    square = np.square(diff) #편차의 제곱
    loss = np.mean(square) #제곱된 편차의 평균
    return loss, diff

def backprop_postproc(G_loss, diff):
    return 2*diff/np.prod(diff.shape) #MSE 미분

In [7]:
init_model_hidden1() #히든층의 크기와 임의의 w,b 설정
for e in range(10001): #순전파와 역전파의 반복
    for i in range(len(x0)):
        output, aux_nn = forward_neuralnet_hidden1(x0[i]) #순전파 진행 :: output은 최종xw+b , aux_nn는 최초입력 x와 히든층에서 ReLU(wx+b)
        loss, aux_pp = forward_postproc(output, y0[i]) # MSE계산 :: loss는 mse값, diff는 편차
        G_loss = 1.0 # 가장 마지막에서 들어가는 값 // 역전파 시작하는 값
        G_output = backprop_postproc(G_loss, aux_pp) # loss의 미분
        backprop_neuralnet_hidden1(G_output, aux_nn) # loss의 미분값을 네트워크에 전달
    if e % 1000==0:
        print(loss)
        

54.25762983971117
0.11147008661219282
0.016830535186428554
0.002583211262319417
0.0004936789344231788
9.320996123891057e-05
1.7771073896680038e-05
3.3581645696713215e-06
6.418997792995645e-07
1.2085274035292968e-07
2.309751647580418e-08


In [8]:
output #최종학습된 가중치 및 편향에 x0를 대입한값 // hypothesis = np.matmul(relu(np.matmul(x0, pm_hidden['w']) + pm_hidden['b']), pm_output['w'])+pm_output['b']
#output[0],output[3]은 0에 가까운값
#output[1],output[2]은 1에 가까운값 으로
#threshold를 0.5로 적용한다면 y와 같은값 출력


array([[0.00015198]])

In [17]:
def pred(x):
    return np.matmul(relu(np.matmul(x,pm_hidden['w'])+pm_hidden['b']),pm_output['w'])+pm_output['b']
for xx in x0:
    print('{} xor {} ==> y = {} : predict : {} '.format(xx[0],xx[1],(pred(xx)>0.5).astype(np.float64)))

0 xor 0 ==>: predict : [0.] 
0 xor 1 ==>: predict : [1.] 
1 xor 0 ==>: predict : [1.] 
1 xor 1 ==>: predict : [0.] 
